<a href="https://colab.research.google.com/github/DishaJillella/CredResolve---Next-best-action-system/blob/main/LightBGM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

cred_resolve_intelligence_challenge_the_next_best_selection_path = kagglehub.competition_download('cred-resolve-intelligence-challenge-the-next-best-selection')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

BASE_PATH = "/kaggle/input/cred-resolve-intelligence-challenge-the-next-best-selection/"

train = pd.read_csv(BASE_PATH + "train.csv")
test  = pd.read_csv(BASE_PATH + "test.csv")

sample_sub = pd.read_csv(BASE_PATH + "sample_submission.csv")

whatsapp = pd.read_csv(BASE_PATH + "whatsapp_activity.csv")
call_placed = pd.read_csv(BASE_PATH + "call_placed.csv")
ai_sms = pd.read_csv(BASE_PATH + "AI_sms_callback.csv")
teleco = pd.read_csv(BASE_PATH + "teleco_call_back.csv")
mobile_app = pd.read_csv(BASE_PATH + "mobile_app_data.csv")
meta = pd.read_csv(BASE_PATH + "metaData.csv")


In [ ]:
import pandas as pd

BASE_PATH = "/kaggle/input/cred-resolve-intelligence-challenge-the-next-best-selection/"

# Load files
train = pd.read_csv(BASE_PATH + "train.csv")
test  = pd.read_csv(BASE_PATH + "test.csv")

meta = pd.read_csv(BASE_PATH + "metaData.csv")
whatsapp = pd.read_csv(BASE_PATH + "whatsapp_activity.csv")
call_placed = pd.read_csv(BASE_PATH + "call_placed.csv")
ai_sms = pd.read_csv(BASE_PATH + "AI_sms_callback.csv")
teleco = pd.read_csv(BASE_PATH + "teleco_call_back.csv")
mobile = pd.read_csv(BASE_PATH + "mobile_app_data.csv")

def inspect_df(name, df, n=5):
    print("="*80)
    print(f"{name.upper()} DATA")
    print("="*80)
    print("\nShape:", df.shape)
    print("\nColumns:")
    for c in df.columns:
        print(" -", c)
    print("\nFirst rows:")
    display(df.head(n))

inspect_df("train", train)
inspect_df("test", test)
inspect_df("meta", meta)
inspect_df("whatsapp", whatsapp)
inspect_df("call_placed", call_placed)
inspect_df("ai_sms", ai_sms)
inspect_df("teleco", teleco)
inspect_df("mobile_app", mobile)


In [ ]:
import numpy as np

train["__is_train"] = 1
test["__is_train"] = 0

test["TARGET"] = np.nan  # explicitly numeric

full = pd.concat([train, test], axis=0, ignore_index=True)


In [ ]:
ACTION_MAP = {
    "ACTION_DIGITAL": "digital",
    "ACTION_BOT": "bot",
    "ACTION_HUMAN_CALL": "human_call",
    "ACTION_FIELD": "field"
}

full["action"] = full["suggested_action"].map(ACTION_MAP)


In [ ]:

whatsapp["sent_at"] = pd.to_datetime(whatsapp["sent_at"], errors="coerce")
whatsapp["read_at"] = pd.to_datetime(whatsapp["read_at"], errors="coerce")

call_placed["start_time"] = pd.to_datetime(call_placed["start_time"], errors="coerce")
teleco["start_time"] = pd.to_datetime(teleco["start_time"], errors="coerce")

mobile["visit_date"] = pd.to_datetime(mobile["visit_date"], errors="coerce")


In [ ]:
meta["state"] = meta["state"].str.lower().str.strip()
meta["dpd_bucket"] = meta["dpd_bucket"].str.strip()
full = full.merge(
    meta,
    on="lead_code",
    how="left"
)
print(full.shape)
full.head()
full.isnull().mean().sort_values(ascending=False).head(10)


In [ ]:
# WhatsApp aggregates
wa_agg = whatsapp.groupby("lead_code").agg(
    whatsapp_sent_cnt = ("sent_at", "count"),
    whatsapp_read_cnt = ("read_at", lambda x: x.notna().sum())
).reset_index()

wa_agg["whatsapp_read_rate"] = (
    wa_agg["whatsapp_read_cnt"] / (wa_agg["whatsapp_sent_cnt"] + 1)
)
full = full.merge(
    wa_agg,
    on="lead_code",
    how="left"
)


In [ ]:
call_agg = call_placed.groupby("lead_code").agg(
    human_call_cnt = ("start_time", "count"),
    human_call_answered = (
        "disposition", lambda x: (x == "ANSWERED").sum()
    ),
    human_call_avg_duration = ("duration", "mean")
).reset_index()

call_agg["human_call_answer_rate"] = (
    call_agg["human_call_answered"] / (call_agg["human_call_cnt"] + 1)
)
full = full.merge(
    call_agg,
    on="lead_code",
    how="left"
)


In [ ]:
bot_agg = teleco.groupby("lead_code").agg(
    bot_call_cnt = ("start_time", "count"),
    bot_call_answered = (
        "disposition", lambda x: (x == "ANSWERED").sum()
    ),
    bot_call_avg_duration = ("duration", "mean")
).reset_index()

bot_agg["bot_call_answer_rate"] = (
    bot_agg["bot_call_answered"] / (bot_agg["bot_call_cnt"] + 1)
)

full = full.merge(
    bot_agg,
    on="lead_code",
    how="left"
)


In [ ]:
field_agg = mobile.groupby("lead_code").agg(
    field_visit_cnt = ("visit_date", "count"),
    field_met_cnt = (
        "result", lambda x: (x == "MET_CUSTOMER").sum()
    )
).reset_index()

field_agg["field_meet_rate"] = (
    field_agg["field_met_cnt"] / (field_agg["field_visit_cnt"] + 1)
)

full = full.merge(
    field_agg,
    on="lead_code",
    how="left"
)


In [ ]:
history_cols = [
    "whatsapp_sent_cnt", "whatsapp_read_rate",
    "human_call_cnt", "human_call_answer_rate",
    "bot_call_cnt", "bot_call_answer_rate",
    "field_visit_cnt", "field_meet_rate"
]

full[history_cols] = full[history_cols].fillna(0)


In [ ]:
def action_hist_rate(row):
    if row["action"] == "digital":
        return row["whatsapp_read_rate"]
    if row["action"] == "human_call":
        return row["human_call_answer_rate"]
    if row["action"] == "bot":
        return row["bot_call_answer_rate"]
    if row["action"] == "field":
        return row["field_meet_rate"]
    return 0

full["action_hist_rate"] = full.apply(action_hist_rate, axis=1)


In [ ]:
full[history_cols + ["action_hist_rate"]].describe()


# **Cost and Intensity Features**


This section adds business-aware features that encode the relative cost and intensity
of each collection action. These features help the model align probability predictions
with ROI without explicitly choosing the action.

In [ ]:
import numpy as np

# Define cost per action
ACTION_COST = {
    "digital": 1,
    "bot": 5,
    "human_call": 20,
    "field": 150
}

# Define intensity per action
ACTION_INTENSITY = {
    "digital": 1,
    "bot": 2,
    "human_call": 3,
    "field": 4
}

# Create features
full["action_cost"] = full["action"].map(ACTION_COST)
full["log_action_cost"] = np.log1p(full["action_cost"])
full["action_intensity"] = full["action"].map(ACTION_INTENSITY)

# Sanity check
full[["action", "action_cost", "action_intensity"]].drop_duplicates().sort_values("action")


# **Model ready dataset prep**

In [ ]:
train_df = full[full["__is_train"] == 1].copy()
test_df  = full[full["__is_train"] == 0].copy()

print(train_df.shape, test_df.shape)


In [ ]:
TARGET_COL = "TARGET"
DROP_COLS = ["id", "lead_code", "__is_train", "suggested_action"]


In [ ]:
categorical_cols = ["action", "dpd_bucket", "state"]

feature_cols = [c for c in train_df.columns
                if c not in DROP_COLS + [TARGET_COL]]

numeric_cols = [c for c in feature_cols if c not in categorical_cols]

print("Categorical:", categorical_cols)
print("Numeric:", numeric_cols)


In [ ]:
X_train = train_df[feature_cols]
y_train = train_df[TARGET_COL].astype(float)

X_test = test_df[feature_cols]


In [ ]:
print(X_train.shape, y_train.shape, X_test.shape)

X_train.isnull().mean().sort_values(ascending=False).head(10)


In [ ]:
X_train = train_df[feature_cols].copy()
X_test  = test_df[feature_cols].copy()


In [ ]:
# Columns where NaN means "no activity"
zero_fill_cols = [
    "field_met_cnt",
    "field_visit_cnt",
    "bot_call_answered",
    "bot_call_cnt",
    "bot_call_avg_duration",
    "human_call_answered",
    "human_call_cnt",
    "human_call_avg_duration",
    "whatsapp_read_cnt",
    "whatsapp_sent_cnt",
]

for col in zero_fill_cols:
    if col in X_train.columns:
        X_train[col] = X_train[col].fillna(0)
        X_test[col] = X_test[col].fillna(0)


# **Train Baseline Model (LightGBM Regressor)**

In [ ]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    random_state=42
)

print(X_tr.shape, X_val.shape)


In [ ]:
# Explicitly cast categorical columns to category dtype
for col in categorical_cols:
    X_tr[col] = X_tr[col].astype("category")
    X_val[col] = X_val[col].astype("category")
    X_test[col] = X_test[col].astype("category")

# Verify dtypes
X_tr.dtypes[categorical_cols]


In [ ]:
import lightgbm as lgb

lgb_model = lgb.LGBMRegressor(
    objective="regression",
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    max_depth=-1,
    min_data_in_leaf=50,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

lgb_model.fit(
    X_tr,
    y_tr,
    eval_set=[(X_val, y_val)],
    eval_metric="rmse",
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=50)
    ]
)


In [ ]:
val_preds = lgb_model.predict(X_val).clip(0, 1)


In [ ]:
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import brier_score_loss

iso = IsotonicRegression(out_of_bounds="clip")
iso.fit(val_preds, y_val)

val_iso = iso.predict(val_preds)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Raw predictions
val_preds = lgb_model.predict(X_val).clip(0, 1)

# Isotonic calibration
from sklearn.isotonic import IsotonicRegression
iso = IsotonicRegression(out_of_bounds="clip")
iso.fit(val_preds, y_val)
val_iso = iso.predict(val_preds)

# RMSE (manual sqrt)
rmse_raw = np.sqrt(mean_squared_error(y_val, val_preds))
rmse_iso = np.sqrt(mean_squared_error(y_val, val_iso))

# MAE
mae_raw = mean_absolute_error(y_val, val_preds)
mae_iso = mean_absolute_error(y_val, val_iso)

rmse_raw, rmse_iso, mae_raw, mae_iso



# **Final Training**

In [ ]:
import lightgbm as lgb

# Ensure categorical dtypes on full data
for col in categorical_cols:
    X_train[col] = X_train[col].astype("category")
    X_test[col] = X_test[col].astype("category")

final_model = lgb.LGBMRegressor(
    objective="regression",
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    max_depth=-1,
    min_data_in_leaf=50,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

final_model.fit(X_train, y_train)


In [ ]:
from sklearn.isotonic import IsotonicRegression
import numpy as np

train_preds = final_model.predict(X_train).clip(0, 1)

iso_final = IsotonicRegression(out_of_bounds="clip")
iso_final.fit(train_preds, y_train)


In [ ]:
test_preds_raw = final_model.predict(X_test).clip(0, 1)
test_preds_cal = iso_final.predict(test_preds_raw)


In [ ]:
test_preds_cal = np.clip(test_preds_cal, 0, 1)


In [ ]:
# Mild cost-aware sharpening
LAMBDA = 0.003  # small, safe value

test_preds_shaped = test_preds_cal * np.exp(-LAMBDA * test_df["action_cost"])
test_preds_shaped = np.clip(test_preds_shaped, 0, 1)


In [ ]:
submission = pd.DataFrame({
    "id": test_df["id"],
    "TARGET": test_preds_cal
})

submission.head()

submission.to_csv("submission.csv", index=False)


In [ ]:
submission["TARGET"].describe()


count    20000.000000
mean         0.397357
std          0.120550
min          0.082503
25%          0.303386
50%          0.346911
75%          0.508099
max          0.787634
Name: TARGET, dtype: float64

Further Testing


# ROS Testing

In [ ]:
ACTION_COST = {
    "digital": 1,
    "bot": 5,
    "human_call": 20,
    "field": 150
}


In [ ]:
# Start from test_df (already exists)
eval_df = test_df.copy()

# Add model predictions
eval_df["pred"] = test_preds_cal  # calibrated predictions

# Action cost mapping
ACTION_COST = {
    "digital": 1,
    "bot": 5,
    "human_call": 20,
    "field": 150
}

eval_df["cost"] = eval_df["action"].map(ACTION_COST)

# Expected recovery proxy
eval_df["expected_recovery"] = eval_df["pred"] * eval_df["total_due"]

# ROI proxy (what evaluator roughly does)
eval_df["roi"] = eval_df["expected_recovery"] - eval_df["cost"]


In [ ]:
chosen = (
    eval_df
    .sort_values("roi", ascending=False)
    .groupby("lead_code", as_index=False)
    .head(1)
)

chosen.shape


(20000, 31)

In [ ]:
total_recovery = chosen["expected_recovery"].sum()
total_cost = chosen["cost"].sum()

roas = total_recovery / total_cost
roas


np.float64(2577.851557838847)

In [ ]:
import numpy as np

eval_df["random_pred"] = np.random.rand(len(eval_df))
eval_df["random_recovery"] = eval_df["random_pred"] * eval_df["total_due"]
eval_df["random_roi"] = eval_df["random_recovery"] - eval_df["cost"]

chosen_random = (
    eval_df
    .sort_values("random_roi", ascending=False)
    .groupby("lead_code", as_index=False)
    .head(1)
)

roas_random = chosen_random["random_recovery"].sum() / chosen_random["cost"].sum()
roas_random


np.float64(2891.8516562310656)

In [ ]:
joblib.dump(final_model, "lgb_model.pkl")
joblib.dump(iso_final, "iso_calibrator.pkl")
joblib.dump(feature_cols, "feature_cols.pkl")
joblib.dump(categorical_cols, "categorical_cols.pkl")


['categorical_cols.pkl']